# Create a Simple Crypto Coin Arbitrage Bot Between Two Exchanges: Bitfinex and Binance

### Purpose of this tutorial
To create an augmented trading tool that will find, and alert you to, arbitrage opportunities across all assets on 2 different exchanges.
    
### What you will be creating:
An augmented trading tool that identifies realtime arbitrage opportunies across 2 exchanges that:
  - can be configured to watch specific assets OR all assets shared between the exchanges
  - monitors both exchanges and all of their assets in real time
  - displays a dataframe of coins with arbitrage opportunies that cointains:
      - name of coin
      - price on bitfinex
      - price on bittrex
      - amount to be 'made' by exploiting the opportunity
      - and the 'recipe' for exploiting the opportunity:
     - e.g recipe: `BUY ETHUSD on BITFINEX and SELL ETHUSD on BINANCE`
  - updates this dataframe, checks for arbitrage opportunies, and prints the dataframe/recipe list every 7 seconds
  - plays a sound if an arbitrage opportunity is found

    
### Prerequistes:
- Python3
- `livedataframe` package: `pip3 install livedataframe`
- public and secret keys from https://app.livedataframe.com/users/new (sign up is free)
- a can-do attitude

    
    
    


## Introduction To Arbitrage

#### What is Arbitrage?
Arbitrage is where you look at the price of an identical asset (in this case crypto coins) on two different exchanges and exploit a price difference (an arbitrage opportunity).

#### How will we be using it?
We will be looking at the price of identical crypto coins on Bitfinex and Binance and determining if an asset is trading at a higher price on a given exchange. We will _not_ be considering transfer/withdrawl fees, or trade/order fees in Part 1 of this tutorial. We will be making a tool that will automatically alert us when one of these opporunties arises.




## Lets Get Started


### Import Required Packages

- If you have not installed livedataframe please do so now: `pip3 install livedataframe`

In [2]:
from livedataframe import LiveExchange, ExchangeInfo # The core packages used by this tutorial
import pandas as pd # import pandas for creating our new tool
from IPython.display import Audio, clear_output # for playing our alert sound and our monitoring tool
import time # for sleeping our tools loop

### Add your LiveDataFrame Api Keys
In the box below, please replace `<YOUR_PUBLIC_KEY_HERE>` and `<YOUR_SECRET_KEY_HERE>` with your actual public and secret LiveDataFrame keys (as strings). 

These Api keys should have been emailed to you when you  when you [signed up](https://app.livedataframe.com/users/new) for LiveDataFrane

In [101]:
PUBLIC_KEY = "<YOUR_PUBLIC_KEY_HERE>"
SECRET_KEY = "<YOUR_SECRET_KEY_HERE>"

## Prepare the symbols we want to watch
Bitfinex has asset/base pairs that Binance does not have, and vice versa. The first thing we have to do is curate a list of asset/base pairs shared between each exchange. 



### Find the Bases (Markets) shared between Bitfinex and Binance

Let's start by getting a list of the base currencies used on each exchange.

In order to do this we will create two functions: one for getting a base from a pair and another for returning a list of bases from a list of pairs.

To get these lists of pairs we will utilize the helper class `ExchangeInfo` and its method `list_symbols_for`. This method will return a list of actively traded _pairs_ for that exchange. Once we have this list of pairs we will have to pull out the portion referencing the base currency.

Conveniently, both binance and bitfinex use the format `QUOTEBASE` (e.g the symbol `ETHUSD` corresponds to a quote asset of `ETH` (ethereum) and a base asset of `USD` (U.S Dollar). So, in order to get the base currency we just have to look at the tail end of the symbol returned. 

**GOTCHA:**
Usually the base currency is the last 3 characters of our symbol (as in the example above); however, there is one base that is 4 letters long: `USDT`. We need to manually check for `USDT` before we take the last 3 characters as the base - in the case of `USDT` we need to take the last 4 characters.

#### Create Functions to Get Base Currencies From a List of Pairs


In [78]:
def get_base_from_pair(pair):
        # USDT is the only 4 letter base currency so we should manually check for it.
        # if found lets manually put it into the base list instead of taking the last 3 characters
        if 'USDT' in pair:
            return 'USDT'  
        else:
            return pair[-3:] # use the last 3 characters as the base currency identifier
            
def get_base_currencies_from_pairs(pairs):
    bases = [] # create empty list to store bases
    for pair in pairs: # iterate through all pairs returned
        base = get_base_from_pair(pair) # get base
        bases.append(base)

    return list(set(bases))# turn list into a set to get unique values and then back into a list

#### Get List of Base Currencies on Bitfinex

In [69]:
bitfinex_pairs = ExchangeInfo.list_symbols_for('bitfinex')
bitfinex_bases = get_base_currencies_from_pairs(bitfinex_pairs)
bitfinex_bases

['ETH', 'USD', 'JPY', 'BTC', 'EUR', 'GBP']

#### Get a List of Base Currencies on Binance

In [70]:
binance_pairs = ExchangeInfo.list_symbols_for('binance')
binance_bases = get_base_currencies_from_pairs(binance_pairs)
binance_bases

['BTC', 'USDT', 'BNB', 'ETH']

#### Get a List of Shared Base Currencies
Now that we have a list of bases for each exchange, let's find out which bases are shared between the two exchanges. These shared bases will be the markets our arbitrage bot will operate in.

In [58]:
shared_bases = list(set(binance_bases).intersection(bitfinex_bases))
shared_bases

['ETH', 'BTC']

At the time of writing there were only two shared bases: `ETH` and `BTC` so those are the only two markets that our arbitrage bot/script will operate in

### Curate the List of Shared Pairs from the Shared Bases (markets)
Now that we have a list of the shared bases (markets) on the two exchanges lets get a list of all the pairs shared between binance and bitfinex with these shared bases.

To start, let's create another function to select pairs with a specific base. This function will take a list of pairs and a specified list of bases to filter by. If a pair has one of the specified bases then it will be included in the output. If not, it will be ignored (filtered).


#### Create Function To Select Only Pairs that have the Specified Base Currencies

In [86]:
def select_pairs_with_specified_base_currencies(specified_base_currencies, pairs):
    filtered_pairs = [] # create empty list to put pairs with specified base_currency into
    
    for specified_base_currency in specified_base_currencies: # itereate through list of specified base currencies
        for pair in pairs: # iterate through pairs
            base = get_base_from_pair(pair)

            if base == specified_base_currency: # check to see if pair has the specified base we are looking for
                filtered_pairs.append(pair) # add pair to filtered pairs list because it has correct base
            
    return filtered_pairs

#### Get Bitfinex Pairs for Shared Bases (Markets)

In [87]:
bitfinex_pairs_with_shared_bases = select_pairs_with_specified_base_currencies(shared_bases, bitfinex_pairs)
bitfinex_pairs_with_shared_bases

['IOTETH',
 'EOSETH',
 'SANETH',
 'OMGETH',
 'BCHETH',
 'NEOETH',
 'ETPETH',
 'QTMETH',
 'AVTETH',
 'EDOETH',
 'DATETH',
 'QSHETH',
 'YYWETH',
 'GNTETH',
 'SNTETH',
 'BATETH',
 'MNAETH',
 'FUNETH',
 'ZRXETH',
 'TNBETH',
 'SPKETH',
 'TRXETH',
 'RCNETH',
 'RLCETH',
 'AIDETH',
 'SNGETH',
 'REPETH',
 'ELFETH',
 'IOSETH',
 'AIOETH',
 'REQETH',
 'RDNETH',
 'LRCETH',
 'WAXETH',
 'DAIETH',
 'CFIETH',
 'AGIETH',
 'BFTETH',
 'MTNETH',
 'ODEETH',
 'ANTETH',
 'DTHETH',
 'MITETH',
 'STJETH',
 'XLMETH',
 'XVGETH',
 'MKRETH',
 'VENETH',
 'KNCETH',
 'POAETH',
 'LYMETH',
 'UTKETH',
 'VEEETH',
 'DADETH',
 'ORSETH',
 'AUCETH',
 'POYETH',
 'FSNETH',
 'CBTETH',
 'LTCBTC',
 'ETHBTC',
 'ETCBTC',
 'RRTBTC',
 'ZECBTC',
 'XMRBTC',
 'DSHBTC',
 'XRPBTC',
 'IOTBTC',
 'EOSBTC',
 'SANBTC',
 'OMGBTC',
 'BCHBTC',
 'NEOBTC',
 'ETPBTC',
 'QTMBTC',
 'AVTBTC',
 'EDOBTC',
 'BTGBTC',
 'DATBTC',
 'QSHBTC',
 'YYWBTC',
 'GNTBTC',
 'SNTBTC',
 'BATBTC',
 'MNABTC',
 'FUNBTC',
 'ZRXBTC',
 'TNBBTC',
 'SPKBTC',
 'TRXBTC',
 'RCNBTC',

#### Get Binance Pairs for Shared Bases (Markets)

In [88]:
binance_pairs_with_shared_bases = select_pairs_with_specified_base_currencies(shared_bases, binance_pairs)
binance_pairs_with_shared_bases

['QTUMETH',
 'EOSETH',
 'SNTETH',
 'BNTETH',
 'BNBETH',
 'OAXETH',
 'DNTETH',
 'MCOETH',
 'ICNETH',
 'WTCETH',
 'LRCETH',
 'OMGETH',
 'ZRXETH',
 'STRATETH',
 'SNGLSETH',
 'BQXETH',
 'KNCETH',
 'FUNETH',
 'SNMETH',
 'NEOETH',
 'IOTAETH',
 'LINKETH',
 'XVGETH',
 'SALTETH',
 'MDAETH',
 'MTLETH',
 'SUBETH',
 'ETCETH',
 'MTHETH',
 'ENGETH',
 'ZECETH',
 'ASTETH',
 'DASHETH',
 'BTGETH',
 'EVXETH',
 'REQETH',
 'VIBETH',
 'HSRETH',
 'TRXETH',
 'POWRETH',
 'ARKETH',
 'YOYOETH',
 'XRPETH',
 'MODETH',
 'ENJETH',
 'STORJETH',
 'VENETH',
 'KMDETH',
 'RCNETH',
 'NULSETH',
 'RDNETH',
 'XMRETH',
 'DLTETH',
 'AMBETH',
 'BCCETH',
 'BATETH',
 'BCPTETH',
 'ARNETH',
 'GVTETH',
 'CDTETH',
 'GXSETH',
 'POEETH',
 'QSPETH',
 'BTSETH',
 'XZCETH',
 'LSKETH',
 'TNTETH',
 'FUELETH',
 'MANAETH',
 'BCDETH',
 'DGDETH',
 'ADXETH',
 'ADAETH',
 'PPTETH',
 'CMTETH',
 'XLMETH',
 'CNDETH',
 'LENDETH',
 'WABIETH',
 'LTCETH',
 'TNBETH',
 'WAVESETH',
 'GTOETH',
 'ICXETH',
 'OSTETH',
 'ELFETH',
 'AIONETH',
 'NEBLETH',
 'BRDETH'

#### Find Shared Pairs (finally)

For each exchange, we have now identified the list of pairs (symbols) that are trading in the markets shared by both exchanges. The last step is to to turn these two lists into one list by finding the shared pairs in these shared markets.

To do this we simply find the intersection of the two lists with some fancy pythoning.

In [104]:
pairs_to_arbitrage = list(set(binance_pairs_with_shared_bases).intersection(bitfinex_pairs_with_shared_bases))

## Create the Arbitrage Tool/Bot

We now have a list of pairs we can hunt for arbitrage opportunities on: `pairs_to_arbitrage`. The next step is build the tool itself. We will start by instantiating two instances of `LiveExchange`: one for Bitfinex and one for Bittrex. 

`LiveExchange` provides us with a dictionary of dataframes that updates every 5 seconds with new data (i.e LiveDataFrames). The keys on this dictionary are the pairs and the values are the LiveDataFrames - there is one key/value and hence one symbol/LiveDataFrame combo for every pair. 

We will dive into `LiveExchange` a little later. For now, let's just create the instances.

### Initialize the LiveExchange Instances for Both Exchanges
We will initialize the LiveExchange instances with a 4 hour lookback period and our list of pairs to arbitrage. This means that for each pair we are watching (from the `pairs_to_arbitrage` list we calculated earlier) there will be a LiveDataFrame with 4 hours of historical data in it.


#### Initialize the LiveExchange Instance for Bitfinex



In [105]:
live_bitfinex = LiveExchange(
    exchange='bitfinex', 
    symbols = pairs_to_arbitrage, 
    public_key = PUBLIC_KEY, 
    secret_key = SECRET_KEY,
    lookback_period = '4h'
)

#### Initialize the LiveExchange Instance for Binance



In [106]:
live_bitfinex = LiveExchange(
    exchange='binance', 
    symbols = pairs_to_arbitrage, 
    public_key = PUBLIC_KEY, 
    secret_key = SECRET_KEY,
    lookback_period = '4h'
)

## Start the LiveExchange Instances

After we have initialized the live exchange instances `live_bitfinex` and `live_binance`, we must start them by calling `.start()` on each instance. Calling `.start()` loads the historical data, subscribes to websockets for each symbol, and then appends each new tick into its own dataframe giving you a live dataframe with a 4 hour window of data. Let's start each LiveExchange instance now.

Please note that, depending on your internet speed, when you first call .start() it can take anywhere from 10 seconds to 3 minutes *per exchange* to load the historical data. This wait period is **only** for the first load up and you only have to do it at the start and never again... so please be patient with this initial load :)


In [108]:
live_bitfinex.start()
live_binance.start()

Seeding With Data... This Could Take a Few Minutes.
Not Authorized. Please Verify Public and Secret Api Keys
***
ERROR! CODE: 401 -- Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/livedataframe/LiveExchange.py", line 67, in start
    self._load_configs_and_messages()
  File "/usr/local/lib/python3.6/site-packages/livedataframe/LiveExchange.py", line 155, in _load_configs_and_messages
    response = self._get('http://%s/api/v1/messages_and_configs' %(self.ip), {"message": 'hello'}).json()
  File "/usr/local/lib/python3.6/site-packages/livedataframe/LiveExchange.py", line 248, in _get
    raise Exception('ERROR! CODE: %s' %(response.status_code))
Exception: ERROR! CODE: 401



NameError: name 'live_bittrex' is not defined

NameError: name 'live_bittrex' is not defined

When you first call `.start()` it can take a little while (anywhere from 10 seconds - 3 minutes depending on your internet speed) to load the initial historical data. Once that load is complete, you will have access to a dictionary of live data frames for every symbol on bittrex. To access one of these live dataframes use the `.symbols` attribute on the `live_bittrex` instance. For example, to access the live df for `BTC-ETH` we would specify `live_bittrex.symbols['BTC-ETH']`. This returns a dataframe that updates every 5 seconds. To see an update, wait 5 seconds and call it again.

Let's go ahead and access the livedataframe for the `BTC-ETH` symbol



# Access Your first Live DataFrame

In [ ]:
live_bittrex.symbols['BTC-ETH'].tail()

# Create Function for Building the Volume Change Monitor

Let's actually start creating our augmented trading tool for monitoring volume changes. This tool will be a dataframe where the indexes will be the symbol names and the volume change over the specified interval will be the column. Let's go ahead and write a function do create this volume tool. This function will take the `LiveExchange` instance (`live_bittrex` in this case) and the specified time interval as inputs. 

In [ ]:
def create_volume_monitor_dataframe(live_exchange, time_interval):
    output = {} # instantiate dictionary to hold our volume indicators
    
#     live_exchange = live_exchange
    count = 0
    for symbol, live_dataframe in live_exchange.symbols.items(): # Loop through every symbol and its corresponding live dataframe in the live_exchange instance
        live_dataframe = live_dataframe.last(time_interval)        
        
        beginning_volume = live_dataframe['Volume'].iloc[0] # volume of asset at the specified time interval "ago" . E.g: if time_interval was '10m' this would be volume 10 minutes ago
        current_volume = live_dataframe['Volume'].iloc[-1] # current volume of the asset
        
        percent_change = ((current_volume - beginning_volume) / beginning_volume) * 100# calculate volume percent change
        
        output[symbol] = percent_change # store percent change
        
        
    volume_monitor_df = pd.DataFrame.from_dict(output, orient = 'index') # create volume monitor dataframe with indexes as symbols and column values as volume percent changes
    volume_monitor_df.columns = [time_interval + " vol % chg"] # add column name to dataframe as the specifed interval
    
    return volume_monitor_df # return the volume monitor dataframe    

# Run Function and Create Volume Monitor DataFrame
Now that we have created a function that builds a new dataframe that adds a "Vol % Change" column for our specified interval accross all bittrex assets, let's see the result. Let's take a look at the volume change for each asset over the last 1 hour and sort our dataframe so the coins with the highest volume changes are at the top

In [ ]:
volume_change_period = '1h' # look at the volume change over 1 hour
create_volume_monitor_dataframe(live_bittrex, volume_change_period).sort_values(by = volume_change_period + " vol % chg", ascending = False)

As we can see, we have just created a dataframe that shows us the percent changes over the last 1 hour for each asset and ranks them by greatest volume change -- pretty cool huh? Now, let's go ahead and implement the volume threshold.

# Add a Volume Threshold Filter
Currently, our tool lists the volume changes for all of the coins on bittrex for a specified time period. There are a lot of coins on bittrex, 291 at the time of writing, so let's filter this list down to what we want. Let's make a function that will take our `volume_monitor_df` and a `volume_threshold` as parameters and then filter our dataframe down to symbols whose volume changes are greater than this threshold.

In [ ]:
def filter_volume_monitor_df_by_threshold(volume_monitor_df, volume_threshold):
    symbols_above_threshold = volume_monitor_df.loc[volume_monitor_df[volume_monitor_df.columns[0]] > volume_threshold] # select the rows from volume monitor dataframe where the volume change is greater than threshold 

    return symbols_above_threshold

# Use the Volume Threshold Filter
Let's go ahead and use this filter function now and filter all symbols that have had a volume change of at LEAST 20% in the last 1 hour.

In [ ]:
volume_monitor_df = create_volume_monitor_dataframe(live_bittrex, '1h')
filter_volume_monitor_df_by_threshold(volume_monitor_df, 20)


# Monitor, and Alert on, Volume Changes in Realtime!
At this point we could keep running these two functions together over and over again and record the changes. This would allow us to 'monitor' the changes, but it wouldn't exactly be automated. Let's put these two functions in a loop that will check our volume changes every 7 seconds and list the coins that have volume changes greater than our threshold over the specified interval. For fun, let's also add the ability to play a cash register sound if our tool discovers volume changes that meet our criteria. We can enable this sound by passing `alert_with_sound = True` into our function.


### Create The Function

In [ ]:
def monitor_volume_changes_and_alert_when_threshold_reached(live_exchange, volume_threshold, time_interval, alert_with_sound = False):
   
    while True:
        volume_monitor_df = create_volume_monitor_dataframe(live_exchange, time_interval)
        coins_above_threshold = filter_volume_monitor_df_by_threshold(volume_monitor_df, volume_threshold)

        if len(coins_above_threshold) > 0: # check to see if there are any coins meeting criteria

            if alert_with_sound: # play sound if alert_with_sound kw param is passed in
                display(Audio("http://thecyberbuddy.com/sounds/CashRegister1.wav", autoplay = True)) # play cash register sound

            clear_output(wait = True) # clear jupyter notebook dataframe output
            
            coins_above_threshold = coins_above_threshold.applymap("{0:.2f}%".format) #  format the % change ouput
            display(coins_above_threshold) # redisplay jupyter notebook dataframe output with updated values

        
        else:
            print('No coins found with volume changes greater than %s%% in the last %s' % (volume_threshold, time_interval))
            
        time.sleep(7) # wait 7 seconds before checking the volumes again



### Run The Function
Now that we have created our master function that will alert us to volume changes that satistfy our criteria. Lets implement it in a real-world scenario: finding coins that are being pumped.

Let's set our `time_interval` to `20s` and our `volume_threshold` to `.10` (a tenth of a percent change). When we run this function we will be alerted if a coin has seen a significant increase in volume in the last 20 seconds. You may want to specify `alert_with_sound = False` after running it a few times to silence the noise. If you do not see any coins, try lowering your volume threshold or increasing your time interval. Happy Hunting! :)

In [1]:
monitor_volume_changes_and_alert_when_threshold_reached(
    live_bittrex, 
    volume_threshold = .10, 
    time_interval = '20s', 
    alert_with_sound = False
)

NameError: name 'monitor_volume_changes_and_alert_when_threshold_reached' is not defined

# Next Steps
- Try monitoring multiple time intervals at the same time and come up with an algorithm for alerting based on these changes:
    - e.g "alert when 10m interval is > 5% AND 1hr interval is >20%"
- Place orders instead of alerting
- Turn the display into a heatmap or better visual output
- Find correlations in the volume changes of different assets
- Use jupyter/ipython widgets to create a GUI for using this tool (instead of typing in parameters)
- Be sure to check out https://github.com/LiveDataFrame/Tutorials for more tutorials utilizing live dataframe